In [11]:
import pandas as pd
import numpy as np
import os
import re

In [12]:
def extract_lhg(filename: str):

    pattern = r"L(\d+)H(\d+)G(\d+)"
    match = re.search(pattern, filename)

    if not match:
        return None
    
    L, H, G = map(int, match.groups())
    return {"L": L, "H": H, "G": G}

In [13]:
def despike_signal(signal: np.ndarray):
    """
    Implementa UNA pasada del método Phase-Space Thresholding de Goring & Nikora (2002)
    con reemplazo cúbico de spikes.

    Flujo:
      0) Remover la media de la señal (trabajar centrado en 0)
      1) Calcular Du y D2u (derivadas centradas, sin dividir por dt)
      2) Calcular desviaciones estándar y Universal Threshold
      3) Calcular ángulo de rotación θ entre u y D2u
      4) Calcular parámetros de las elipses en las 3 proyecciones
      5) Detectar spikes (puntos fuera del elipsoide en alguna proyección)
      6) Reemplazar cada evento de spikes con un polinomio cúbico ajustado
         a hasta 12 puntos buenos a cada lado.

    Devuelve:
      - clean_signal: señal despikeada (ya con media repuesta)
      - spike_mask: máscara booleana con True donde se detectaron spikes
    """

    # ---------- Paso 0: remover la media ----------
    # El paper indica que antes de cualquier algoritmo se remueve la media.
    # Trabajamos con una señal centrada en 0 (u) para que el cluster en fase
    # sea simétrico y la estadística sea más estable.
    signal = np.asarray(signal, dtype=float)
    mean_u = np.nanmean(signal)
    u = signal - mean_u  # señal centrada

    n = u.size
    if n < 5:
        # Con muy pocos datos no tiene sentido aplicar el método
        clean_signal = signal.copy()
        spike_mask = np.zeros_like(signal, dtype=bool)
        return clean_signal, spike_mask

    # ---------- Paso 1: derivadas Du y D2u ----------
    # Usamos las fórmulas de diferencias centradas del paper:
    #   Du_i  = (u_{i+1} - u_{i-1}) / 2
    #   D2u_i = (Du_{i+1} - Du_{i-1}) / 2
    # Importante: NO dividimos por dt, así u, Du y D2u están en el mismo orden
    # de magnitud (cm/s) y la geometría del elipsoide es razonable.
    Du = np.empty_like(u)
    D2u = np.empty_like(u)

    Du[1:-1] = (u[2:] - u[:-2]) / 2.0
    # En los bordes no hay vecinos simétricos, copiamos el valor del interior
    # más cercano. El impacto estadístico en 10.000 muestras es despreciable.
    Du[0] = Du[1]
    Du[-1] = Du[-2]

    D2u[1:-1] = (Du[2:] - Du[:-2]) / 2.0
    D2u[0] = D2u[1]
    D2u[-1] = D2u[-2]

    # ---------- Paso 2: desviaciones estándar + Universal Threshold ----------
    # Usamos desviación estándar "poblacional" (ddof=0) porque la teoría
    # del Universal Threshold usa N (no N-1) y trabajamos con la serie completa.
    su = np.nanstd(u, ddof=0)
    sDu = np.nanstd(Du, ddof=0)
    sD2u = np.nanstd(D2u, ddof=0)

    # Si todo es constante (σ ~ 0), no hay nada que despikear
    if su == 0 and sDu == 0 and sD2u == 0:
        clean_signal = signal.copy()
        spike_mask = np.zeros_like(signal, dtype=bool)
        return clean_signal, spike_mask

    # Universal Threshold: λ_U = sqrt(2 ln n)
    # Esto da la escala máxima esperada para una serie normal de tamaño n.
    lambda_U = np.sqrt(2.0 * np.log(n))

    # ---------- Paso 3: ángulo de rotación θ entre u y D2u ----------
    # El paper define θ ≈ atan( Σ u_i D2u_i / Σ u_i^2 ).
    # Usamos arctan2 por robustez numérica (maneja signos y x≈0).
    sum_u_D2u = np.nansum(u * D2u)
    sum_u2 = np.nansum(u * u)

    if sum_u2 == 0.0:
        # Si Σu^2 = 0, la señal es casi constante → no tiene sentido un ángulo;
        # tomamos θ = 0 como caso degenerado seguro.
        theta = 0.0
    else:
        theta = np.arctan2(sum_u_D2u, sum_u2)

    cos_t = np.cos(theta)
    sin_t = np.sin(theta)

    # ---------- Paso 4: parámetros de las elipses en las 3 proyecciones ----------
    # 4.A) Proyección (u, Du):
    #   - semieje mayor  = λ_U * σ_u  (dirección u)
    #   - semieje menor  = λ_U * σ_Du (dirección Du)
    a_u_Du = lambda_U * su
    b_u_Du = lambda_U * sDu

    # 4.B) Proyección (Du, D2u):
    #   - semieje mayor  = λ_U * σ_Du   (dirección Du)
    #   - semieje menor  = λ_U * σ_D2u  (dirección D2u)
    a_Du_D2u = lambda_U * sDu
    b_Du_D2u = lambda_U * sD2u

    # 4.C) Proyección (u, D2u) rotada:
    # Usamos las ecuaciones del paper para obtener los semiejes a y b a partir de:
    #   (λ_U σ_u)^2   = a^2 cos^2θ + b^2 sin^2θ
    #   (λ_U σ_D2u)^2 = a^2 sin^2θ + b^2 cos^2θ
    A = (lambda_U * su) ** 2
    B = (lambda_U * sD2u) ** 2

    cos2 = cos_t * cos_t
    sin2 = sin_t * sin_t
    # Este denominador es proporcional a cos(2θ). Si es ~0, el sistema se vuelve
    # mal condicionado y preferimos no jugar con números raros.
    denom = cos2 * cos2 - sin2 * sin2

    if np.isclose(denom, 0.0):
        # Caso degenerado: usamos directamente A y B como si no hubiera rotación.
        a_u_D2u = np.sqrt(max(A, 0.0))
        b_u_D2u = np.sqrt(max(B, 0.0))
    else:
        # Resolución explícita del sistema para a^2 y b^2.
        a2 = (A * cos2 - B * sin2) / denom
        b2 = (B * cos2 - A * sin2) / denom

        # Por si el redondeo numérico deja algo levemente negativo, lo recortamos a 0.
        a2 = max(a2, 0.0)
        b2 = max(b2, 0.0)

        a_u_D2u = np.sqrt(a2)
        b_u_D2u = np.sqrt(b2)

    # ---------- Paso 5: detección de spikes ----------
    # Estrategia: un punto es "bueno" si está dentro de TODAS las elipses
    # en las 3 proyecciones. Si cae fuera en alguna → se marca como spike.
    inside_all = np.ones_like(u, dtype=bool)

    # 5.A) Proyección (u, Du) — elipse estándar centrada en (0,0):
    #      (x/a)^2 + (y/b)^2 <= 1 → punto dentro.
    if a_u_Du > 0 and b_u_Du > 0:
        x = u
        y = Du
        inside_u_Du = (x / a_u_Du) ** 2 + (y / b_u_Du) ** 2 <= 1.0
        inside_all &= inside_u_Du

    # 5.B) Proyección (Du, D2u)
    if a_Du_D2u > 0 and b_Du_D2u > 0:
        x = Du
        y = D2u
        inside_Du_D2u = (x / a_Du_D2u) ** 2 + (y / b_Du_D2u) ** 2 <= 1.0
        inside_all &= inside_Du_D2u

    # 5.C) Proyección rotada (u, D2u)
    if a_u_D2u > 0 and b_u_D2u > 0:
        # Primero rotamos (u, D2u) al sistema de ejes principales de la elipse:
        #   x' =  u cosθ + D2u sinθ
        #   y' = -u sinθ + D2u cosθ
        x_prime = u * cos_t + D2u * sin_t
        y_prime = -u * sin_t + D2u * cos_t

        inside_u_D2u = (x_prime / a_u_D2u) ** 2 + (y_prime / b_u_D2u) ** 2 <= 1.0
        inside_all &= inside_u_D2u

    # Todo lo que NO está dentro en las 3 proyecciones → spike
    spike_mask = ~inside_all

    # Si no hay spikes, devolvemos la señal original (solo sumamos media)
    if not np.any(spike_mask):
        clean_signal = signal.copy()
        return clean_signal, spike_mask

    # ---------- Paso 6: reemplazo cúbico sobre eventos de spikes ----------
    # Trabajamos sobre u_clean (señal centrada); luego sumamos la media de vuelta.
    u_clean = u.copy()
    idx = np.arange(n)

    # Obtenemos índices de todos los puntos marcados como spike
    spike_indices = np.where(spike_mask)[0]

    # Agrupamos spikes contiguos en "eventos" (bloques)
    events = []
    start = spike_indices[0]
    prev = spike_indices[0]

    for k in spike_indices[1:]:
        if k == prev + 1:
            # Sigue el bloque
            prev = k
        else:
            # Terminó un bloque y empieza otro
            events.append((start, prev))
            start = prev = k
    events.append((start, prev))  # último bloque

    for (s, e) in events:
        # Para cada evento [s, e], buscamos puntos buenos a izquierda y derecha.
        left_good = idx[(idx < s) & (~spike_mask)]
        right_good = idx[(idx > e) & (~spike_mask)]

        # Tomamos hasta 12 puntos buenos a cada lado (lo recomendado en el paper).
        left_sel = left_good[-12:]
        right_sel = right_good[:12]

        # Concat de índices que usaremos para el ajuste del polinomio
        fit_idx = np.concatenate([left_sel, right_sel])

        # Si no hay suficientes puntos buenos para ajustar un polinomio razonable:
        if fit_idx.size < 4:
            # --- Fallbacks sencillos ---
            spike_range = np.arange(s, e + 1)

            if left_sel.size > 0 and right_sel.size > 0:
                # Hay un punto bueno antes y uno después → interpolación lineal.
                x0 = left_sel[-1]
                x1 = right_sel[0]
                y0 = u_clean[x0]
                y1 = u_clean[x1]
                u_clean[spike_range] = np.interp(spike_range, [x0, x1], [y0, y1])
            elif left_sel.size > 0:
                # Solo hay puntos buenos a la izquierda → extendemos el último valor bueno.
                u_clean[spike_range] = u_clean[left_sel[-1]]
            elif right_sel.size > 0:
                # Solo hay puntos buenos a la derecha → extendemos el primer valor bueno.
                u_clean[spike_range] = u_clean[right_sel[0]]
            # Si no hay nada en ninguno de los lados (caso muy raro), no tocamos el spike.
            continue

        # Ajuste de polinomio:
        # Usamos np.polyfit sobre los puntos buenos (fit_idx, u_clean[fit_idx]).
        x_fit = fit_idx.astype(float)
        y_fit = u_clean[fit_idx]

        # Por diseño deberíamos tener >= 4 puntos → cúbico.
        # Igual dejamos un check defensivo para evitar grados inválidos.
        degree = 3
        if fit_idx.size <= 3:
            degree = fit_idx.size - 1  # 1 → lineal, 2 → cuadrático, etc.

        coeffs = np.polyfit(x_fit, y_fit, degree)

        # Evaluamos el polinomio en todos los índices del evento.
        spike_range = np.arange(s, e + 1, dtype=float)
        u_clean[s:e + 1] = np.polyval(coeffs, spike_range)

    # Volvemos a sumar la media para volver al nivel original de la señal.
    clean_signal = u_clean + mean_u

    return clean_signal, spike_mask


In [14]:
VELOCITY_COLUMNS = [
    "V1/X/E(cm/s)",
    "V2/Y/N(cm/s)",
    "V3/Z/U(cm/s)",
]

def process_csv_file(filename: str, output_suffix: str = "_clean"):

    print(f"Procesando archivo: {filename}")
    
    data_info = extract_lhg(filename)
    print("Info LHG:", data_info)

    file_path = './raw_data/' + filename
    
    df = pd.read_csv(file_path, sep="\t")
    
    for col in VELOCITY_COLUMNS:

        if col not in df.columns:
            print(f"Algo mal. columna {col} no encontrada en {filename}, se omite.")
            continue
        
        print(f"  → Despiking en columna: {col}")
        signal = df[col].values
        
        clean_signal, spike_mask = despike_signal(signal)
        print("Spikes:", spike_mask.sum())
        
        clean_col = col + "_clean"
        mask_col = col + "_is_spike"
        
        df[clean_col] = clean_signal
        df[mask_col] = spike_mask
    
    base, ext = os.path.splitext(filename)
    output_name = f"./clean_data/{base}{output_suffix}{ext}"
    
    df.to_csv(output_name, index=False, sep="\t")
    print(f"  ✅ Guardado como: {output_name}\n")


In [15]:
csv_files = [file for file in os.listdir(os.getcwd() + '/raw_data') if file.endswith(".csv")]

print(f"Se encontraron {len(csv_files)} archivos CSV.\n")

for i, filename in enumerate(csv_files, start=1):
    print(f"[{i}/{len(csv_files)}]")
    process_csv_file(filename)

Se encontraron 30 archivos CSV.

[1/30]
Procesando archivo: L220H12G16_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 12, 'G': 16}
  → Despiking en columna: V1/X/E(cm/s)
Spikes: 53
  → Despiking en columna: V2/Y/N(cm/s)
Spikes: 77
  → Despiking en columna: V3/Z/U(cm/s)
Spikes: 30
  ✅ Guardado como: ./clean_data/L220H12G16_Probe_1_(A837F)._clean.csv

[2/30]
Procesando archivo: L220H15G16_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 15, 'G': 16}
  → Despiking en columna: V1/X/E(cm/s)
Spikes: 18
  → Despiking en columna: V2/Y/N(cm/s)
Spikes: 35
  → Despiking en columna: V3/Z/U(cm/s)
Spikes: 34
  ✅ Guardado como: ./clean_data/L220H15G16_Probe_1_(A837F)._clean.csv

[3/30]
Procesando archivo: L220H18G14_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 18, 'G': 14}
  → Despiking en columna: V1/X/E(cm/s)
Spikes: 33
  → Despiking en columna: V2/Y/N(cm/s)
Spikes: 29
  → Despiking en columna: V3/Z/U(cm/s)
Spikes: 49
  ✅ Guardado como: ./clean_data/L220H18G14_Probe_1_(A837F)._clean.csv

[4/30]
Pro